In [1]:
import findspark, pyspark
findspark.find()

'C:\\PROGRA~1\\apache\\spark-2.4.7-bin-hadoop2.7'

# RDD 
- 특정 칼럼의 고유한 값의 개수를 카운트
- A로 시작하는 단어들 찾기
- 스크린에 결과 출력

### 트랜스폼 함수
- .map(lambda v: (v, 1))를 이용 스파크가 A를 포함하는 단어를 모음
- .filter(lambda val: val.startwith('A))를 사용 A로 시작하는 단어 필터링
- .reduceByKey(operator.add())를 호출

### 액션 함수
- .collect() 함수를 이용하여 컨텍스트로 데이터를 가져옴.

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = SparkConf().setAppName('RDD1').setMaster('local[2]')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

In [2]:
data = sc.parallelize([
    ('Amber', 22), ('Alfred', 23), ('Skye', 4), ('Albert', 12), ('Amber', 9)
])

In [3]:
data_from_file = sc.textFile('./VS14MORT.txt.gz', 4)

In [5]:
type(data_from_file)

pyspark.rdd.RDD

In [6]:
type(data)

pyspark.rdd.RDD

In [4]:
data_heterogenous = sc.parallelize([
    ('Ferrari', 'fast'), {'Porsche': 100000}, ['Spain', 'visited', 4504]
]).collect()

In [8]:
data_heterogenous

[('Ferrari', 'fast'), {'Porsche': 100000}, ['Spain', 'visited', 4504]]

In [9]:
data_heterogenous[1]['Porsche']

100000

In [6]:
data_from_file.take(1)

['                   1                                          2101  M1087 432311  4M4                2014U7CN                                    I64 238 070   24 0111I64                                                                                                                                                                           01 I64                                                                                                  01  11                                 100 601']

In [7]:
def extractInformation(row):
    import re
    import numpy as np

    selected_indices = [
         2,4,5,6,7,9,10,11,12,13,14,15,16,17,18,
         19,21,22,23,24,25,27,28,29,30,32,33,34,
         36,37,38,39,40,41,42,43,44,45,46,47,48,
         49,50,51,52,53,54,55,56,58,60,61,62,63,
         64,65,66,67,68,69,70,71,72,73,74,75,76,
         77,78,79,81,82,83,84,85,87,89
    ]

    '''
        Input record schema
        schema: n-m (o) -- xxx
            n - position from
            m - position to
            o - number of characters
            xxx - description
        1. 1-19 (19) -- reserved positions
        2. 20 (1) -- resident status
        3. 21-60 (40) -- reserved positions
        4. 61-62 (2) -- education code (1989 revision)
        5. 63 (1) -- education code (2003 revision)
        6. 64 (1) -- education reporting flag
        7. 65-66 (2) -- month of death
        8. 67-68 (2) -- reserved positions
        9. 69 (1) -- sex
        10. 70 (1) -- age: 1-years, 2-months, 4-days, 5-hours, 6-minutes, 9-not stated
        11. 71-73 (3) -- number of units (years, months etc)
        12. 74 (1) -- age substitution flag (if the age reported in positions 70-74 is calculated using dates of birth and death)
        13. 75-76 (2) -- age recoded into 52 categories
        14. 77-78 (2) -- age recoded into 27 categories
        15. 79-80 (2) -- age recoded into 12 categories
        16. 81-82 (2) -- infant age recoded into 22 categories
        17. 83 (1) -- place of death
        18. 84 (1) -- marital status
        19. 85 (1) -- day of the week of death
        20. 86-101 (16) -- reserved positions
        21. 102-105 (4) -- current year
        22. 106 (1) -- injury at work
        23. 107 (1) -- manner of death
        24. 108 (1) -- manner of disposition
        25. 109 (1) -- autopsy
        26. 110-143 (34) -- reserved positions
        27. 144 (1) -- activity code
        28. 145 (1) -- place of injury
        29. 146-149 (4) -- ICD code
        30. 150-152 (3) -- 358 cause recode
        31. 153 (1) -- reserved position
        32. 154-156 (3) -- 113 cause recode
        33. 157-159 (3) -- 130 infant cause recode
        34. 160-161 (2) -- 39 cause recode
        35. 162 (1) -- reserved position
        36. 163-164 (2) -- number of entity-axis conditions
        37-56. 165-304 (140) -- list of up to 20 conditions
        57. 305-340 (36) -- reserved positions
        58. 341-342 (2) -- number of record axis conditions
        59. 343 (1) -- reserved position
        60-79. 344-443 (100) -- record axis conditions
        80. 444 (1) -- reserve position
        81. 445-446 (2) -- race
        82. 447 (1) -- bridged race flag
        83. 448 (1) -- race imputation flag
        84. 449 (1) -- race recode (3 categories)
        85. 450 (1) -- race recode (5 categories)
        86. 461-483 (33) -- reserved positions
        87. 484-486 (3) -- Hispanic origin
        88. 487 (1) -- reserved
        89. 488 (1) -- Hispanic origin/race recode
     '''

    record_split = re\
        .compile(
            r'([\s]{19})([0-9]{1})([\s]{40})([0-9\s]{2})([0-9\s]{1})([0-9]{1})([0-9]{2})' + 
            r'([\s]{2})([FM]{1})([0-9]{1})([0-9]{3})([0-9\s]{1})([0-9]{2})([0-9]{2})' + 
            r'([0-9]{2})([0-9\s]{2})([0-9]{1})([SMWDU]{1})([0-9]{1})([\s]{16})([0-9]{4})' +
            r'([YNU]{1})([0-9\s]{1})([BCOU]{1})([YNU]{1})([\s]{34})([0-9\s]{1})([0-9\s]{1})' +
            r'([A-Z0-9\s]{4})([0-9]{3})([\s]{1})([0-9\s]{3})([0-9\s]{3})([0-9\s]{2})([\s]{1})' + 
            r'([0-9\s]{2})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([\s]{36})([A-Z0-9\s]{2})([\s]{1})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([\s]{1})([0-9\s]{2})([0-9\s]{1})' + 
            r'([0-9\s]{1})([0-9\s]{1})([0-9\s]{1})([\s]{33})([0-9\s]{3})([0-9\s]{1})([0-9\s]{1})')
    try:
        rs = np.array(record_split.split(row))[selected_indices]
    except:
        rs = np.array(['-99'] * len(selected_indices))
    return rs
#     return record_split.split(row)

In [8]:
data_from_file_conv = data_from_file.map(extractInformation)
data_from_file_conv.map(lambda row: row).take(1)

[array(['1', '  ', '2', '1', '01', 'M', '1', '087', ' ', '43', '23', '11',
        '  ', '4', 'M', '4', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'I64 ',
        '238', '070', '   ', '24', '01', '11I64  ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '01',
        'I64  ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], dtype='<U40')]

# Transformations
## .map()
map() 함수 : RDD의 각 엘리먼트에 적용된다.

In [14]:
data_2014 = data_from_file_conv.map(lambda row: int(row[16]))

In [15]:
data_2014.take(10)

[2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, -99]

## .filter()
이 함수를 이용해서 데이터셋으로부터 특정 조건에 맞는 엘리먼트를 선택 할 수 있다.

In [16]:
data_filtered = data_from_file_conv.filter(lambda row: row[5] == 'F' and row[21] == '0')

In [17]:
data_filtered.count()

6

In [18]:
data_filtered1 = data_from_file_conv.filter(lambda row: row[16] == '2014' and row[21] == '0')

In [19]:
data_filtered1.count()

22

## .flatMap()
.flatMap() 함수는 .map()함수와 비슷, 리스트가 아닌 평면화된 결과를 리턴

In [20]:
data_2014_flat = data_from_file_conv.flatMap(lambda row: (row[16], int(row[16]) + 1))

In [21]:
data_2014_flat.take(10)

['2014', 2015, '2014', 2015, '2014', 2015, '2014', 2015, '2014', 2015]

## .distinct()
이 함수는 특정 컬럼에서 중복된 값을 제거, 고유한 값을 리스트로 반환

In [22]:
distinct_gender = data_from_file_conv.map(lambda row: row[5]).distinct().collect()

In [23]:
distinct_gender

['M', 'F', '-99']

## .sample()
자원을 많이 사용하는 함수, 데이터를 섞는 연산을 포함하기 때문, 꼭 필요한 경우만 사용해야 함.
- 첫번째 : 중복 허용 여부
- 두번째 : 리턴 할 데이터셋과 전체 데이터셋 같은 크기 비율
- 세번째 : 임의의 숫자를 생성하기위한 시드값

In [24]:
fraction = 0.1
data_sample = data_from_file_conv.sample(False, fraction, 55)

In [25]:
data_sample.take(1)

[array(['1', '  ', '3', '1', '01', 'F', '1', '086', ' ', '43', '23', '11',
        '  ', '4', 'W', '6', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'F03 ',
        '175', '111', '   ', '37', '01', '11F03  ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '01',
        'F03  ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], dtype='<U40')]

In [26]:
print('Original dataset: {0}, sample: {1}'.format(data_from_file_conv.count(), data_sample.count()))

Original dataset: 2631171, sample: 263231


## .leftOuterJoin()
왼쪽을 기준으로 두개의 데이터셋을 합침

In [27]:
rdd1 = sc.parallelize([('a', 1), ('b', 4), ('c', 10)])
rdd2 = sc.parallelize([('a', 4), ('a', 1), ('b', '6'), ('d', 15)])

In [28]:
rdd3 = rdd1.leftOuterJoin(rdd2)
rdd3.take(5)

[('b', (4, '6')), ('c', (10, None)), ('a', (1, 4)), ('a', (1, 1))]

In [29]:
rdd4 = rdd1.join(rdd2)
rdd4.collect()

[('b', (4, '6')), ('a', (1, 4)), ('a', (1, 1))]

In [30]:
rdd5 = rdd1.intersection(rdd2)
rdd5.collect()

[('a', 1)]

## .repartition()
테이터셋 재파티션하여 데이터가 나눠지는 파티션의 갯구가 바뀜, 부하가 많이 가는 작업

In [31]:
rdd1 = rdd1.repartition(4)
len(rdd1.glom().collect())

4

# Actions
액션은 테이터셋에서 스케줄된 태스크를 실행한다.

## .take()
.map()함수처럼 .take()함수는 중요한 함수이다.
하나의 파티션에서 가장 위에 있는 n행을 리턴한다.

In [32]:
data_first = data_from_file_conv.take(1)
data_first

[array(['1', '  ', '2', '1', '01', 'M', '1', '087', ' ', '43', '23', '11',
        '  ', '4', 'M', '4', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'I64 ',
        '238', '070', '   ', '24', '01', '11I64  ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '01',
        'I64  ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], dtype='<U40')]

## .takeSample()
- 첫번째 : 중복 허용 여부
- 두번째 : 리턴되는 데이터 갯수
- 세번째 : 랜덤 시드 값

In [33]:
data_take_sample = data_from_file_conv.takeSample(False, 1, 55)
data_take_sample

[array(['2', '  ', '3', '1', '03', 'M', '1', '068', ' ', '39', '19', '09',
        '  ', '5', 'S', '4', '2014', 'U', '7', 'B', 'N', ' ', ' ', 'I251',
        '215', '063', '   ', '21', '04', '11I469 ', '21J189 ', '31I500 ',
        '41I251 ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '04',
        'I251 ', 'I469 ', 'I500 ', 'J189 ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], dtype='<U40')]

## .count()
.count()함수는 전체 데이터셋을 드라이버로 옮기지않는다.

In [10]:
!conda list pyspark

# packages in environment at C:\ProgramData\Anaconda3:
#
# Name                    Version                   Build  Channel
pyspark                   3.0.1                    pypi_0    pypi


In [9]:
data_from_file_conv.count()

2631171

In [11]:
len(data_from_file_conv.collect()) # 데이텃셋 전체를 드라이버로 리턴한다.

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5.0 failed 1 times, most recent failure: Lost task 0.0 in stage 5.0 (TID 6, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:584)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:575)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:410)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$class.foreach(Iterator.scala:891)
	at org.apache.spark.InterruptibleIterator.foreach(InterruptibleIterator.scala:28)
	at scala.collection.generic.Growable$class.$plus$plus$eq(Growable.scala:59)
	at scala.collection.mutable.ArrayBuffer.$plus$plus$eq(ArrayBuffer.scala:104)
	at scal

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 1709)
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "C:\ProgramData\Anaconda3\lib\socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "C:\ProgramData\Anaconda3\lib\socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "C:\ProgramData\Anaconda3\lib\socketserver.py", line 720, in __init__
    self.handle()
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyspark\accumulators.py", line 269, in handle
    poll(accum_updates)
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyspark\accumulators.py", line 241, in poll
    if func():
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyspark\accumulators.py", line 245, in accum_updates
   

In [ ]:
data_key = sc.parallelize([('a', 4), ('b', 3), ('c', 2), ('a', 8), ('b', 2), ('b', 1),('d', 3)], 4)
# 키 값을 기반으로 리듀스 하기
data_key.reduceByKey(lambda x, y: x + y).collect()

In [ ]:
data_key.countByKey().items()

## .saveAsTextFile()
각 파티션을 분리된 파일에 저장

In [ ]:
data_key.saveAsTextFile('./data_key.txt')

In [ ]:
def parseInput(row):
    import re
    pattern = re.compile(r'\(\'([a-z])\', ([0-9])\)')
    row_split = pattern.split(row)
    return (row_split[1], int(row_split[2]))

In [ ]:
data_key_reread = sc.textFile('./data_key.txt').map(parseInput)

In [ ]:
data_key_reread.collect()

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 1118)
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "C:\ProgramData\Anaconda3\lib\socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "C:\ProgramData\Anaconda3\lib\socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "C:\ProgramData\Anaconda3\lib\socketserver.py", line 720, in __init__
    self.handle()
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyspark\accumulators.py", line 269, in handle
    poll(accum_updates)
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyspark\accumulators.py", line 241, in poll
    if func():
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyspark\accumulators.py", line 245, in accum_updates
   